In [1]:
import pandas as pd
import os
import pickle
import bmra_prep
import bmra_prep.pathway_activity.prediction

In [16]:
data_dir = "/home/jyang/Phd_project/project_GBM/gbm_Scripts/gbm_Scripts_bmra/01_outputs_2020"
#info_dir = data_dir

out_dir = "02_outputs_2020"

os.makedirs(out_dir, exist_ok = True)


# Load Data

In [17]:
# load metdadata dict and extract used elements
with open(os.path.join(data_dir, "metadata.pickle"), "rb") as f:
    all_metadata = pickle.load(f)

n_modules = all_metadata["n_modules"]
n_genes = all_metadata["n_genes"]
n_experiments = all_metadata["n_experiments"]

modules = all_metadata["modules"]
exp_ids = all_metadata["exp_ids"]
genes = all_metadata["genes"]

In [18]:
# load data
L1000_df = pd.read_csv(
    os.path.join(data_dir, "L1000_Data_norm_data.csv"),
    index_col = 0,
)

x = L1000_df.values
x.shape

(978, 119)

In [19]:
# load doses and perturbation matrix
inhib_conc_matrix = pd.read_csv(
    os.path.join(data_dir, "inhib_conc_annotated.csv"),
    index_col = 0,
).values

ic50_matrix = pd.read_csv(
    os.path.join(data_dir, "ic50_annotated.csv"),
    index_col = 0,
).values

# gamma_matrix = pd.read_csv(
#     os.path.join(data_dir, "gamma_annotated.csv"),
#     index_col = 0,
# ).values

pert_matrix = pd.read_csv(
    os.path.join(data_dir, "pert_annotated.csv"),
    index_col = 0,
).values

In [20]:
# y_true = (1 + gamma_matrix * inhib_conc_matrix / ic50_matrix) / (1 + inhib_conc_matrix / ic50_matrix)

y_true = 1 / (1 + inhib_conc_matrix / ic50_matrix)

display(y_true.shape)
y_true

(11, 119)

array([[1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       ...,
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 0.86139986, 0.94914478,
        0.98245337]])

## Run models

In [21]:
a_coeffs = bmra_prep.pathway_activity.prediction.predict_coeffs(
    x, y_true, pert_matrix, 200_000, 10, 10, 10, 100)

In [22]:
a_coeffs_df = pd.DataFrame(a_coeffs, index = modules, columns = genes)
a_coeffs_df.to_csv(os.path.join(out_dir, "a_coeffs.csv"))
#a_coeffs_df = pd.read_csv(os.path.join(out_dir,'a_coeffs.csv'),index_col=0)
#a_coeffs = a_coeffs_df.values
display(a_coeffs_df.astype(bool).sum(axis='columns'))
display(a_coeffs_df)

CDK1        978
CDK2        978
CDK4_6      978
PI3K        978
ERK         978
Hypoxia     978
PDGFR       978
Aurora      978
Estrogen    978
EGFR        978
p53         978
dtype: int64

,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,ACAT2,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
CDK1,-0.000007,8.362808e-06,0.000003,0.000011,3.192382e-06,0.000005,-0.000044,6.345355e-06,3.136534e-07,-8.886089e-06,...,-0.000015,1.248052e-06,0.000003,-0.000029,0.000008,-5.303100e-06,8.032515e-06,-8.212541e-06,0.000006,-0.000003
CDK2,0.000036,1.708604e-05,0.000018,0.000006,-1.350189e-06,0.000015,-0.000013,-6.463573e-06,-4.205920e-06,3.367534e-06,...,-0.000013,-4.476217e-07,0.000013,-0.000048,0.000012,-9.414331e-06,-1.624674e-05,-6.950553e-06,0.000005,0.000016
CDK4_6,-0.000003,5.047952e-06,0.000003,0.000009,8.427580e-06,0.000022,0.000011,-1.215887e-06,1.038544e-05,3.427910e-07,...,-0.000004,2.937645e-06,0.000001,-0.000012,0.000001,-2.521478e-06,-2.065584e-08,2.063973e-05,0.000027,0.000032
PI3K,-0.000034,7.072603e-06,-0.000006,-0.000011,1.594308e-06,0.000019,-0.000008,1.042729e-05,1.564086e-05,-5.054812e-06,...,0.000003,1.940894e-06,-0.000004,0.000004,-0.000007,-1.168014e-01,1.395822e-05,5.793878e-07,-0.000007,-0.000008
ERK,-0.000014,-6.403036e-07,0.000007,-0.000031,1.048693e-05,-0.000003,-0.000001,4.116754e-06,9.326349e-06,2.249637e-05,...,0.000010,9.070319e-06,-0.000012,0.000018,-0.000023,-6.983518e-06,-3.332840e-06,6.240598e-07,0.000004,-0.000019
Hypoxia,0.000007,-4.341473e-06,-0.000005,-0.000007,-1.499896e-05,-0.000020,0.000041,-4.082711e-05,-5.489724e-06,-1.785047e-05,...,0.000009,1.836687e-06,0.000009,0.000006,0.000019,4.942695e-06,-1.091502e-05,1.639150e-05,-0.000015,0.000004
PDGFR,-0.000003,-3.505456e-06,-0.000001,0.000011,7.168995e-06,-0.000007,0.000012,1.890650e-05,8.390702e-06,-1.775755e-05,...,-0.000004,2.721833e-05,0.000003,-0.000010,-0.000016,-2.545006e-05,2.685094e-05,-8.418003e-06,0.000027,-0.000010
Aurora,-0.000008,9.352905e-06,-0.000016,-0.000002,9.842264e-07,0.000003,0.000006,8.872981e-07,1.084395e-05,-6.061754e-06,...,0.000008,2.247063e-05,0.000004,0.000010,-0.000004,-5.710247e-06,3.919479e-06,2.326490e-05,0.000007,-0.000012
Estrogen,-0.000017,-2.856441e-05,0.000013,-0.000006,2.910607e-06,0.000011,-0.000011,-4.137166e-06,-6.305215e-06,-2.896903e-05,...,-0.000005,-1.136661e-06,0.000013,0.000017,-0.000042,-7.228371e-07,1.548071e-05,1.681985e-05,0.000020,0.000026
EGFR,-0.000017,-1.893335e-06,0.000002,0.000028,1.810340e-05,0.000012,-0.000007,-1.326891e-05,2.785559e-05,-3.528863e-07,...,-0.000005,-3.929368e-07,0.000007,0.000005,-0.000007,-3.189765e-06,2.376291e-05,1.270523e-05,-0.000005,0.000004


In [23]:
#pathway_activity = a_coeffs @ x
#pathway_activity.shape

In [24]:
R_global = bmra_prep.pathway_activity.calc_global_response_from_pathway_activity(
    bmra_prep.pathway_activity.calc_pathway_activity(x,a_coeffs),
    modules,
    L1000_df.columns
)
R_global_df = R_global.dataframe
display(R_global_df)

,ASG002_GI1_24H:A10,ASG002_GI1_24H:A11,ASG002_GI1_24H:A12,ASG002_GI1_24H:A19,ASG002_GI1_24H:A20,ASG002_GI1_24H:A21,ASG002_GI1_24H:B10,ASG002_GI1_24H:B11,ASG002_GI1_24H:B12,ASG002_GI1_24H:B13,...,MOAR010_GI1_24H:J05,MOAR010_GI1_24H:J06,MOAR010_GI1_24H:J22,MOAR010_GI1_24H:J23,MOAR010_GI1_24H:J24,MOAR010_GI1_24H:K08,MOAR010_GI1_24H:K09,MOAR011_GI1_24H:F07,MOAR011_GI1_24H:F08,MOAR011_GI1_24H:F09
CDK1,-0.532456,-0.090838,-0.000091,0.002085,-0.165900,-0.121836,-0.068890,-0.324622,-0.246149,-0.447347,...,0.010588,0.087241,-0.291894,-0.010809,-0.191546,-0.152258,-0.030956,-0.325711,-0.075607,0.065381
CDK2,-0.005915,0.058437,-0.005472,-0.468645,-0.258947,-0.051079,0.083799,-0.149857,0.069219,0.036831,...,-0.089429,0.014765,-0.007504,-0.235402,0.066590,0.036492,-0.111734,0.021896,0.011417,0.041343
CDK4_6,-1.020350,-0.156489,0.006785,-0.309805,-0.362326,-0.206513,-0.273633,-0.376785,-0.431937,-1.081409,...,0.123113,-0.029467,-0.008531,0.122665,-0.161915,-0.206933,-0.269666,-0.194807,-0.201761,0.131028
PI3K,-1.980469,-1.511715,-1.282948,-1.765735,-1.240682,-0.539564,-0.150972,-0.257132,-0.615737,-0.564920,...,0.144486,-0.325722,-0.181422,-0.156620,-0.412223,-0.142093,-0.242985,-0.738465,-0.335861,0.070843
ERK,-1.069782,-0.434094,-0.327457,-0.627189,-0.324659,-0.664645,-0.421138,-0.976800,-0.589117,-1.109856,...,-0.065764,0.098499,-0.047844,0.108805,0.041457,-0.089113,-0.085854,-0.305833,-0.370252,-0.017460
Hypoxia,-0.254387,0.023142,0.058141,0.041423,-0.035411,-0.038288,0.084564,-0.109133,-0.036479,-0.157980,...,-0.116061,-0.009096,0.055604,-0.090556,-0.062510,-0.037085,0.081561,-0.022819,-0.004053,-0.022677
PDGFR,-1.047116,0.085648,0.249960,-0.016258,-0.287235,-0.056367,0.180483,-0.500696,0.003939,-0.280822,...,0.174566,-0.001951,0.210924,0.135909,0.031303,0.232381,0.105209,-0.087007,-0.096901,0.006698
Aurora,-0.176946,0.165538,0.098505,0.043972,-0.156947,-0.192212,-0.368580,-0.472288,0.026675,-0.881248,...,0.098747,0.135528,0.186694,0.251682,0.001502,0.193990,-0.003241,-0.354541,-0.150168,0.353166
Estrogen,-0.059273,-0.011432,-0.115573,0.024551,0.167143,-0.594219,-0.040831,0.054088,-0.049902,0.112370,...,0.133559,0.177735,0.177713,0.137463,-0.492810,0.212787,-0.405344,-0.401177,0.000818,-0.041655
EGFR,-0.397070,-0.016259,0.172429,-0.192615,-0.109157,0.116712,0.128445,-0.615233,0.143768,-1.271298,...,0.054318,-0.351044,-0.453752,-0.124671,-0.112722,-0.202544,-0.215714,-0.202577,-0.156586,0.036562


In [25]:
R_global_df.to_csv(os.path.join(out_dir, "R_global_annotated.csv"))
display(R_global_df)

,ASG002_GI1_24H:A10,ASG002_GI1_24H:A11,ASG002_GI1_24H:A12,ASG002_GI1_24H:A19,ASG002_GI1_24H:A20,ASG002_GI1_24H:A21,ASG002_GI1_24H:B10,ASG002_GI1_24H:B11,ASG002_GI1_24H:B12,ASG002_GI1_24H:B13,...,MOAR010_GI1_24H:J05,MOAR010_GI1_24H:J06,MOAR010_GI1_24H:J22,MOAR010_GI1_24H:J23,MOAR010_GI1_24H:J24,MOAR010_GI1_24H:K08,MOAR010_GI1_24H:K09,MOAR011_GI1_24H:F07,MOAR011_GI1_24H:F08,MOAR011_GI1_24H:F09
CDK1,-0.532456,-0.090838,-0.000091,0.002085,-0.165900,-0.121836,-0.068890,-0.324622,-0.246149,-0.447347,...,0.010588,0.087241,-0.291894,-0.010809,-0.191546,-0.152258,-0.030956,-0.325711,-0.075607,0.065381
CDK2,-0.005915,0.058437,-0.005472,-0.468645,-0.258947,-0.051079,0.083799,-0.149857,0.069219,0.036831,...,-0.089429,0.014765,-0.007504,-0.235402,0.066590,0.036492,-0.111734,0.021896,0.011417,0.041343
CDK4_6,-1.020350,-0.156489,0.006785,-0.309805,-0.362326,-0.206513,-0.273633,-0.376785,-0.431937,-1.081409,...,0.123113,-0.029467,-0.008531,0.122665,-0.161915,-0.206933,-0.269666,-0.194807,-0.201761,0.131028
PI3K,-1.980469,-1.511715,-1.282948,-1.765735,-1.240682,-0.539564,-0.150972,-0.257132,-0.615737,-0.564920,...,0.144486,-0.325722,-0.181422,-0.156620,-0.412223,-0.142093,-0.242985,-0.738465,-0.335861,0.070843
ERK,-1.069782,-0.434094,-0.327457,-0.627189,-0.324659,-0.664645,-0.421138,-0.976800,-0.589117,-1.109856,...,-0.065764,0.098499,-0.047844,0.108805,0.041457,-0.089113,-0.085854,-0.305833,-0.370252,-0.017460
Hypoxia,-0.254387,0.023142,0.058141,0.041423,-0.035411,-0.038288,0.084564,-0.109133,-0.036479,-0.157980,...,-0.116061,-0.009096,0.055604,-0.090556,-0.062510,-0.037085,0.081561,-0.022819,-0.004053,-0.022677
PDGFR,-1.047116,0.085648,0.249960,-0.016258,-0.287235,-0.056367,0.180483,-0.500696,0.003939,-0.280822,...,0.174566,-0.001951,0.210924,0.135909,0.031303,0.232381,0.105209,-0.087007,-0.096901,0.006698
Aurora,-0.176946,0.165538,0.098505,0.043972,-0.156947,-0.192212,-0.368580,-0.472288,0.026675,-0.881248,...,0.098747,0.135528,0.186694,0.251682,0.001502,0.193990,-0.003241,-0.354541,-0.150168,0.353166
Estrogen,-0.059273,-0.011432,-0.115573,0.024551,0.167143,-0.594219,-0.040831,0.054088,-0.049902,0.112370,...,0.133559,0.177735,0.177713,0.137463,-0.492810,0.212787,-0.405344,-0.401177,0.000818,-0.041655
EGFR,-0.397070,-0.016259,0.172429,-0.192615,-0.109157,0.116712,0.128445,-0.615233,0.143768,-1.271298,...,0.054318,-0.351044,-0.453752,-0.124671,-0.112722,-0.202544,-0.215714,-0.202577,-0.156586,0.036562
